In [1]:
import os
%pwd

'f:\\Project_2\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'f:\\Project_2'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CrawlDataConfigure:
    root_dir: Path
    data_path: Path
    url: str
    ids_url: list
    max_products: int
    max_comments: int
    use_crawl_data: bool


In [5]:
import os
from Project_2.constants import *
from Project_2.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfigureManager:
    def __init__(self,
                config_filepath: Path = CONFIG_FILE_PATH,
                params_filepath: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_crawl_data_config(self) -> CrawlDataConfigure:
        crawldata = self.config.crawl_data
        params = self.params
        create_directories([crawldata.root_dir])

        crawl_data_config = CrawlDataConfigure(
            root_dir = Path(crawldata.root_dir),
            data_path = Path(crawldata.data_path),
            url = params.PARAMS_CRAWL_DATA.URL,
            ids_url = params.PARAMS_CRAWL_DATA.URL_IDS,
            max_comments= params.PARAMS_CRAWL_DATA.MAX_COMMENTS,
            max_products= params.PARAMS_CRAWL_DATA.MAX_PRODCUTS,
            use_crawl_data= params.USE_CRAWL_DATA
        )
        return crawl_data_config



In [11]:
import requests
import time
import random
import pandas as pd

class CrawlData:
    def __init__(self, config: CrawlDataConfigure):
        self.config = config
        self.headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
                        'Referer': config.url
                        }

    def get_product_ids_from_cat(self, cat_id, max_products):
        product_list = []
        page = 1
        seen_ids = set()
        print(f"--- Bắt đầu lấy danh sách sản phẩm từ Category: {cat_id} ---")

        while len(product_list) < max_products:
            url = f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=48&category={cat_id}&page={page}"
            try:
                response = requests.get(url, headers=self.headers)
                if response.status_code == 200:
                    data = response.json()
                    items = data.get('data', [])
                    if not items:break
                    for item in items:
                        if len(product_list) > max_products:break
                        raw_id = item.get('id')
                        p_id = str(raw_id)

                        # KIỂM TRA TRÙNG LẶP DỰA TRÊN ID
                        # (ID là số nên hash được, không bị lỗi)
                        if p_id in seen_ids:
                            continue

                        seen_ids.add(p_id)
                        product_list.append({
                            'id': raw_id,
                            'name': item.get('name', 'No Name')
                        })
                    print(f"Đã tìm thấy {len(product_list)} sản phẩm (Page {page})")
                    page += 1
                    time.sleep(1)
                else:
                    print(f"Lỗi tải danh mục: {response.status_code}")
                    break
            except Exception as e:
                print(f"Lỗi: {e}")
                break
        return product_list

    def get_comment_from_product_ids(self, product_id, max_comments):
        comment_datas = []
        page = 1

        while len(comment_datas) < max_comments:
            url = f"https://tiki.vn/api/v2/reviews?product_id={product_id}&limit=20&page={page}"
            try:
                response = requests.get(url, headers=self.headers)
                if response.status_code == 200:
                    data = response.json()
                    reviews = data.get('data', [])
                    if not reviews: break

                    for rev in reviews:
                        if len(comment_datas) > max_comments: break

                        comment_datas.append({
                            "product_id" : product_id,
                            "user_name": rev.get("created_by", {}).get('full_name'),
                            'rating': rev.get('rating'),
                            'content': rev.get('content'),
                            'created_at': rev.get('created_at')
                        })

                    page += 1
                    time.sleep(random.uniform(0.5, 1.5))
                else:
                    break
            except:
                break

        return comment_datas

    def crawldata(self):
        if self.config.use_crawl_data:
            ALL_DATA = []

            for cat_id in self.config.ids_url:
                # Bước 1: Lấy danh sách sản phẩm (Hàm đã sửa)
                products = self.get_product_ids_from_cat(cat_id, self.config.max_products)

                # Bước 2: Lấy comment (Giữ nguyên code cũ)
                for idx, prod in enumerate(products):
                    print(f"[{idx+1}/{len(products)}] Đang cào comment cho ID {prod['id']}: {prod['name'][:30]}...")
                    comments = self.get_comment_from_product_ids(prod['id'], self.config.max_comments)

                    for c in comments:
                        c['product_name'] = prod['name']
                        ALL_DATA.append(c)

            if ALL_DATA:
                df = pd.DataFrame(ALL_DATA)
                df.to_csv(self.config.data_path, index=False)
                print("\nHoàn tất! Kiểm tra file tiki_reviews_8322.csv")
        else:
            print("Not crawl data")
            return


In [ ]:
try:
    config = ConfigureManager()
    crawl_data_config = config.get_crawl_data_config()
    crawl_data = CrawlData(config=crawl_data_config)
    crawl_data.crawldata()

except Exception as e:
    raise e

[2026-02-01 00:16:11,193: INFO: common: yaml file: configs\config.yaml loaded successfully]
[2026-02-01 00:16:11,195: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-01 00:16:11,196: INFO: common: created directory at: artifacts]
[2026-02-01 00:16:11,197: INFO: common: created directory at: artifacts/crawl_data]
--- Bắt đầu lấy danh sách sản phẩm từ Category: 8322 ---
Đã tìm thấy 40 sản phẩm (Page 1)
Đã tìm thấy 80 sản phẩm (Page 2)
Đã tìm thấy 101 sản phẩm (Page 3)
[1/101] Đang cào comment cho ID 278997061: Sách Tô Màu Phát Triển Trí Não...
[2/101] Đang cào comment cho ID 278967562: Monster - Deluxe Edition –  [C...
[3/101] Đang cào comment cho ID 278901566: Sách Đời Gió Bụi...
[4/101] Đang cào comment cho ID 278879928: Sách Nếu Như Không Thể Nói Nếu...
[5/101] Đang cào comment cho ID 278874643: Sách Đại Pháp Sư Của Thư Viện ...
[6/101] Đang cào comment cho ID 278856619: Sách Cô Bé Hàng Xóm Và Bốn Viê...
[7/101] Đang cào comment cho ID 278838097: Sách Tính Bản Ác (M